#  Cross-sell recommendations for E-commerce
#### Github repositoty: https://github.com/celis/cross-sell-recommendations-for-ecommerce


## A project submitted to the AWS Marketplace Developer Challenge
#### website: https://awsmarketplaceml.devpost.com


This notebook can run locally. Credentials and configuration for the different services need to be set on the 'configs/config.json' file of the repository.    

This notebook fetches transactional data from any Woocommerce Shop using the Woocommerce Rest API, then moves the data to an S3 bucket. The AWS Sagemaker Batch Transform Job retrieves this data from the S3 bucket and uses it to train a [Mphasis-Mphasis-HyperGraf-Market-Basket-Analysis](https://aws.amazon.com/marketplace/pp/Mphasis-Mphasis-HyperGraf-Market-Basket-Analysis/prodview-y6z3fk7gvudgs) model from the AWS Marketplace.   The model outputs product recommendations based on patterns extracted from previous customer purchases. These recommendations can be used in the online shop for cross-selling purposes.  In order to integrate these recommendations to the online shop, the notebook also uses the Woocommerce Rest API to upload the recommendations to the MySql database of WordPress.

After running the full notebook, we will have populated the 'cross_sell_ids' field of our MySQL database (which is empty by default).  Woocommerce provides several options to visualize these product recommendations as sliders on Product or Checkout pages.    This notebook therefore provides a fully fledged solution to automatize the generation of product recommendations for small and medium size merchants. 

In [27]:
import os
import pandas as pd
import boto3

from sagemaker import ModelPackage
import sagemaker as sagemaker

from src.configuration import Configuration
from src.services import S3, Woocommerce

In [18]:
# Configure access to AWS and WordPress

config = Configuration('configs/config.json').parameters

s3 = S3(region_name = config['s3']['region_name'],
        access_key =  config['aws_keys']["access_key"],
       secret_key = config['aws_keys']["secret_key"],
       bucket = config['s3']["bucket"])

woocommerce = Woocommerce(url=config['woocommerce']["url"],
                         consumer_key=config['woocommerce']["consumer_key"],
                         consumer_secret=config['woocommerce']["consumer_secret"])

In [19]:
# Communicate with the MySQL WordPress database via the Woocommerce Rest API
# Downloading transactional data from Woocommerce

woocommerce.download_data('data/input_data.csv')

In [ ]:
# Uploading transactional data to S3

s3.upload('data/input_data.csv','input_data.csv')


# Configuring a batch transform job with a model from AWS Marketplace

model_package_arn = config['batch_transform_job']['model_package_arn']

iam = boto3.client('iam')
role = iam.get_role(RoleName=config['batch_transform_job']['role_name'])['Role']['Arn']

sagemaker_session = sagemaker.Session()


model = ModelPackage(model_package_arn=model_package_arn,
                    role = role,
                    sagemaker_session = sagemaker_session)


# output path where the recommendations are going to be stored
output_path = 's3://' + os.path.join(config['s3']['bucket'],'output')

transformer = model.transformer(instance_count=config['batch_transform_job']['instance_count'], 
                                instance_type=config['batch_transform_job']['instance_type'],
                                output_path=output_path)


# input path where the transactional data used for training the model is located
input_path = 's3://' + os.path.join(config["s3"]["bucket"], config["s3"]["input_file"])

transformer.transform(input_path, content_type='text/csv')

## Model training and producing the output with recommendations

transformer.wait()

In [13]:
# Recommendations produced by the model are stored in S3. Donwloading them:

bucket_folder = transformer.output_path.rsplit('/')[3]
key = os.path.join(bucket_folder, config["s3"]["input_file"].split('/')[-1]+'.out')

s3.download(key, 'data/output_data.csv')

In [14]:
# loading and visualizing the output recommendations

output_df  = pd.read_csv('data/output_data.csv')
output_df  = output_df.drop('Unnamed: 0',1)
out_final = output_df
out_final.head()

,Item in cart,Recommendation,Item Support,Support,Confidence,Lift,Leverage,Conviction
0,RED ORCHID,GREEN ORCHID,0.249443,0.133630,0.535714,3.340774,0.093630,1.808463
1,RED ORCHID,BONSAI PINE,0.249443,0.075724,0.303571,2.163549,0.040724,1.234424
2,RED ORCHID,SWEET WILLIAM,0.249443,0.071269,0.285714,2.729483,0.045159,1.253452
3,RED ORCHID,WHITE ORCHID,0.249443,0.064588,0.258929,2.004464,0.032366,1.175088
4,RED ORCHID,SUNFLOWER,0.249443,0.057906,0.232143,1.966644,0.028462,1.148599


In [15]:
## We push the recommendations to our online shop using the Woocommerce Rest API

woocommerce.upload_recommendations('data/output_data.csv')